In [1]:
from scripts.model_utils import Utils
import torch
from transformers import RobertaForTokenClassification, RobertaTokenizerFast, TrainingArguments, Trainer
from scripts.Reader import obtain_dataset, obtain_label_list

print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    device = torch.device('cuda')
print("Current Device:", torch.cuda.current_device(), torch.cuda.get_device_name(torch.cuda.current_device()))

CUDA available: True
Current Device: 0 NVIDIA GeForce RTX 4070 Ti


In [2]:
# Load tokenizer and model
label_list, label2id, id2label = obtain_label_list("TempEval3")
model_name = 'roberta-large'
tokenizer = RobertaTokenizerFast.from_pretrained(model_name, add_prefix_space=True)
model = RobertaForTokenClassification.from_pretrained(model_name, num_labels=len(label_list), label2id=label2id, id2label=id2label)

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
training_args = TrainingArguments(
    output_dir="./results/Timex3-NER",
    logging_dir="./logs/Timex3-NER",
    eval_strategy="steps",
    save_strategy="steps",
    logging_steps=100,
    num_train_epochs=1,
    save_total_limit=1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    learning_rate=5e-5,
    load_best_model_at_end=True,
    metric_for_best_model="f1"
)

In [5]:
utils = Utils(tokenizer, label_list)
tokenized_datasets = utils.tokenize_datasets(obtain_dataset("TempEval3"))

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Tokenizing datasets...


Map:   0%|          | 0/10117 [00:00<?, ? examples/s]

Map:   0%|          | 0/101 [00:00<?, ? examples/s]

Map:   0%|          | 0/1125 [00:00<?, ? examples/s]

In [6]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 10117
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 101
    })
    eval: Dataset({
        features: ['tokens', 'ner_tags', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1125
    })
})

In [10]:
timex3_ner = Trainer(
    model=model,
    args=training_args,
    compute_metrics=utils.compute_metrics,
    data_collator=utils.data_collator,
    tokenizer=tokenizer,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["eval"],
)

C:\Users\Harry\AppData\Local\Temp\ipykernel_13324\173198253.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  timex3_ner = Trainer(


In [11]:
timex3_ner.train()

OutOfMemoryError: CUDA out of memory. Tried to allocate 32.00 MiB. GPU 0 has a total capacity of 11.99 GiB of which 0 bytes is free. Of the allocated memory 26.06 GiB is allocated by PyTorch, and 209.50 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [10]:
timex3_ner.evaluate(tokenized_datasets["test"])

{'eval_loss': 0.030355069786310196,
 'eval_precision': 0.8291956305858987,
 'eval_recall': 0.8590534979423868,
 'eval_f1': 0.8438605356240525,
 'eval_runtime': 60.4278,
 'eval_samples_per_second': 28.679,
 'eval_steps_per_second': 0.91,
 'epoch': 3.0}

In [13]:
timex3_ner.save_model("./results/Timex3-NER/final_model")